<a href="https://colab.research.google.com/github/ajaykm000563/Agency.AI/blob/main/Text_Summerization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# (Run this in a Colab code cell; leading ! runs shell commands)
!pip install --upgrade pip
!pip install transformers datasets accelerate huggingface_hub sentencepiece evaluate


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 31.9 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2


In [2]:
from datasets import load_dataset

# 1) Load the dataset from Hugging Face Datasets hub
ds = load_dataset("cnn_dailymail", "3.0.0")   # you can replace with "xsum" or your dataset id


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

3.0.0/train-00000-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

3.0.0/train-00001-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

3.0.0/train-00002-of-00003.parquet:   0%|          | 0.00/259M [00:00<?, ?B/s]

3.0.0/validation-00000-of-00001.parquet:   0%|          | 0.00/34.7M [00:00<?, ?B/s]

3.0.0/test-00000-of-00001.parquet:   0%|          | 0.00/30.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

In [3]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline

model_name = "t5-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# quick demo: use the pretrained model to summarize one article (before fine-tuning)
summarizer = pipeline("summarization", model=model, tokenizer=tokenizer, device=0)  # device=0 tries GPU
sample = ds["validation"][10]["article"]
res = summarizer(sample, max_length=60, min_length=20, do_sample=False)[0]["summary_text"]
print("PRETRAINED MODEL SUMMARY (example):\n", res)


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Device set to use cuda:0
Both `max_new_tokens` (=256) and `max_length`(=60) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


PRETRAINED MODEL SUMMARY (example):
 this page includes the show transcript . use the transcript to help students with reading comprehension and vocabulary . at the bottom of the page, comment for a chance to be mentioned on the next CNN Student News. you must be a teacher or a student age 13 or older to request a mention on the CNN Student news Roll Call.


In [4]:
# simple settings to keep memory small
max_input_length = 256
max_target_length = 64

def preprocess_batch(batch):
    # 1) Prepare inputs: add "summarize: " prefix (T5 style)
    inputs = ["summarize: " + a for a in batch["article"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True, padding="max_length")

    # 2) Prepare target summaries (labels)
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(batch["highlights"], max_length=max_target_length, truncation=True, padding="max_length")

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Apply to a small subset so we can print and keep memory low
small = ds["train"].select(range(100))   # first 100 examples
tokenized = small.map(preprocess_batch, batched=True, remove_columns=small.column_names)


Map:   0%|          | 0/100 [00:00<?, ? examples/s]

/usr/local/lib/python3.12/dist-packages/transformers/tokenization_utils_base.py:4034: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [13]:
import os
os.environ["WANDB_DISABLED"] = "true"   # <--- turn off wandb logs

from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments, DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

training_args = Seq2SeqTrainingArguments(
    output_dir="./t5-sum-demo",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    logging_steps=10,
    fp16=True,
    report_to="none"     # <--- THIS disables wandb and others
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized,
    data_collator=data_collator,
    tokenizer=tokenizer,   # This warning is OK
)

train_result = trainer.train()
print("Training complete!")

/tmp/ipython-input-32747824.py:18: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


Step,Training Loss
10,3.600000
20,3.408000
30,3.261700
40,2.680700
50,2.778500
60,2.718600
70,2.449600


Training complete!


In [14]:
trainer.save_model("./t5-sum-demo")
tokenizer.save_pretrained("./t5-sum-demo")

# Show saved files (so you can imagine the files are stored)
import os
print("Saved files:", os.listdir("./t5-sum-demo"))


Saved files: ['runs', 'generation_config.json', 'checkpoint-75', 'tokenizer.json', 'checkpoint-25', 'tokenizer_config.json', 'spiece.model', 'special_tokens_map.json', 'config.json', 'training_args.bin', 'model.safetensors']


In [15]:
from transformers import pipeline

# Reload pipeline from saved folder to be sure we use the fine-tuned weights
summarizer = pipeline("summarization", model="./t5-sum-demo", tokenizer="./t5-sum-demo", device=0)

# pick one test article (line-by-line) and show everything
example = ds["validation"][5]
gen = summarizer(example["article"], max_length=80, min_length=20, do_sample=False)[0]["summary_text"]
print("\nGENERATED SUMMARY:\n", gen)
print("\nREFERENCE SUMMARY:\n", example["highlights"])


Device set to use cuda:0
Token indices sequence length is longer than the specified maximum sequence length for this model (1103 > 512). Running this sequence through the model will result in indexing errors
Both `max_new_tokens` (=256) and `max_length`(=80) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)



GENERATED SUMMARY:
 ,,,,,, and we all have reason to be proud of Gabby Schilling, he says., congratulating his daughter, Gabby, on being accepted to Salve Regina University, where she'll play softball . Schilling mowed down a group of Twitter trolls who made the mistake of tweeting vulgar and sexually-explicit comments about Schilling's daughter .

REFERENCE SUMMARY:
 Ruben Navarrette: Schilling deserves praise for taking on online haters for offensive comments about his daughter .
Navarrette: In protecting his child, Schilling set a model for parenting and taught us a lesson about social media .


In [18]:
import evaluate
rouge = evaluate.load("rouge")

# use small subset to keep it fast
test_small = ds["validation"].select(range(20))
preds = []
refs = []
for ex in test_small:
    out = summarizer(ex["article"], max_length=80, min_length=20, do_sample=False)[0]["summary_text"]
    preds.append(out)
    refs.append(ex["highlights"])

res = rouge.compute(predictions=preds, references=refs)
# print nicely rounded scores
print({k: round(v*100, 3) for k, v in res.items()})


Both `max_new_tokens` (=256) and `max_length`(=80) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=256) and `max_length`(=80) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=256) and `max_length`(=80) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=256) and `max_length`(=80) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both

{'rouge1': np.float64(26.738), 'rouge2': np.float64(8.979), 'rougeL': np.float64(18.772), 'rougeLsum': np.float64(22.003)}


In [17]:
!pip install rouge_score

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24987 sha256=59505ebb89536528e1f4be1e6e47ea055998c5f80d1f95268b0f2d16783865fa
  Stored in directory: /root/.cache/pip/wheels/85/9d/af/01feefbe7d55ef5468796f0c68225b6788e85d9d0a281e7a70
Successfully built rouge_score


In [19]:
paragraph = """One hot summer day when there had been no rain for months and all the ponds and rivers had dried up. A thirsty crow was searching for water. At last, he spotted a pitcher of cool water in a garden and flew down to take a drink, but when he put his head into the neck of the picture, it was only half full, and the crow could not reach the water.
The poor crow knew that if he did not get a drink, soon, he would die of thirst. He had to find some way of getting to the water in the picture as he looked around, wondering what to do. He saw some pebbles on the path and he had an idea, he picked up a pebble in his beak and dropped it into the pitcher. The water level rose a little. The bird got another pebble and dropped it in the water. A little more. The crow worked very hard, dropping more and more bubbles into the picture until the water was almost at the top, at last, the bird was able to reach the water, and he drank and drank until he could drink no more. His clever idea had saved his life."""

In [20]:
gen = summarizer("summarize: " + paragraph, max_length=80, min_length=20, do_sample=False)[0]["summary_text"]
print("summary of this poem : " + gen)

Both `max_new_tokens` (=256) and `max_length`(=80) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


summary of this poem : a thirsty crow spotted a pitcher of cool water in a garden and flew down to take a drink . but when he put his head into the neck of the picture, it was only half full . the bird was able to reach the water, and drank until he could drink no more .
